In [1]:
import numpy as np
import pandas as pd
from src.dataloader_ import *
from src.network_ import *
from src.utils import *

import os
import sys
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer

train_series_dir = "../../inputs/series_train.parquet/"
test_series_dir = "../../inputs/series_test.parquet/"

data_dic_path = "../../inputs/data_dictionary.csv"
sample_submission_path = "../../inputs/sample_submission.csv"
train_path = "../../inputs/train.csv"
test_path = "../../inputs/test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample_submission = pd.read_csv(sample_submission_path)
data_dic = pd.read_csv(data_dic_path)

import os
import random

import numpy as np
import torch


def seed_torch(seed=1029):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


nb_name = os.path.basename(os.getcwd())  # notebook name
seed_torch(seed=42)

In [2]:
survey_cols = [
    "PCIAT-PCIAT_01",
    "PCIAT-PCIAT_02",
    "PCIAT-PCIAT_03",
    "PCIAT-PCIAT_04",
    "PCIAT-PCIAT_05",
    "PCIAT-PCIAT_06",
    "PCIAT-PCIAT_07",
    "PCIAT-PCIAT_08",
    "PCIAT-PCIAT_09",
    "PCIAT-PCIAT_10",
    "PCIAT-PCIAT_11",
    "PCIAT-PCIAT_12",
    "PCIAT-PCIAT_13",
    "PCIAT-PCIAT_14",
    "PCIAT-PCIAT_15",
    "PCIAT-PCIAT_16",
    "PCIAT-PCIAT_17",
    "PCIAT-PCIAT_18",
    "PCIAT-PCIAT_19",
    "PCIAT-PCIAT_20",
]

for col in survey_cols:
    train[col] = train[col] / 5

train["PCIAT-PCIAT_Total"] = train["PCIAT-PCIAT_Total"] / 100

In [3]:
train[survey_cols]

,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20
0,1.0,0.8,0.8,0.0,0.8,0.0,0.0,0.8,0.0,0.0,0.8,0.0,0.8,0.8,0.8,0.8,0.8,0.8,0.4,0.8
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.4,0.4,0.2,0.4,0.2,0.2,0.4,0.2,0.2,0.2,0.0,0.2,0.2,0.2,0.0,0.4,0.4,0.2,0.2
3,0.8,0.4,0.8,0.0,1.0,0.2,0.0,0.6,0.4,0.4,0.6,0.0,0.6,0.0,0.0,0.6,0.8,0.6,0.8,0.2
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,0.6,0.6,0.6,0.4,0.6,0.4,0.4,0.4,0.4,0.2,0.4,0.0,0.4,0.0,0.2,0.0,0.4,0.2,0.2,0.0
3956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3957,1.0,1.0,0.6,0.0,1.0,0.2,0.0,0.4,0.0,0.4,0.2,0.0,0.2,0.6,0.0,0.0,0.2,0.2,0.0,0.2
3958,0.4,0.2,0.2,0.2,0.0,0.0,0.0,0.2,0.2,0.2,0.4,0.0,0.2,0.2,0.4,0.2,0.2,0.2,0.2,0.2


In [4]:
set(train.columns) - set(test.columns)

{'PCIAT-PCIAT_01',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_03',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_05',
 'PCIAT-PCIAT_06',
 'PCIAT-PCIAT_07',
 'PCIAT-PCIAT_08',
 'PCIAT-PCIAT_09',
 'PCIAT-PCIAT_10',
 'PCIAT-PCIAT_11',
 'PCIAT-PCIAT_12',
 'PCIAT-PCIAT_13',
 'PCIAT-PCIAT_14',
 'PCIAT-PCIAT_15',
 'PCIAT-PCIAT_16',
 'PCIAT-PCIAT_17',
 'PCIAT-PCIAT_18',
 'PCIAT-PCIAT_19',
 'PCIAT-PCIAT_20',
 'PCIAT-PCIAT_Total',
 'PCIAT-Season',
 'sii'}

In [5]:
feature_imputer = KNNImputer(n_neighbors=5)
sii_imputer = KNNImputer(n_neighbors=5)

numeric_cols = test.select_dtypes(include=["float64", "int64"]).columns
numeric_feature_cols = numeric_cols.copy()
# numeric_feature_cols = numeric_feature_cols.drop("sii")

numeric_sii_cols = train.select_dtypes(include=["float64", "int64"]).columns

sii_inputed = sii_imputer.fit_transform(train[numeric_sii_cols])
feature_imputer.fit(test[numeric_feature_cols])
feature_inputed = feature_imputer.fit_transform(train[numeric_feature_cols])

train_imputed = pd.DataFrame(feature_inputed, columns=numeric_feature_cols)

for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]

train = train_imputed

sii_impute = pd.DataFrame(sii_inputed, columns=numeric_sii_cols)

for col in survey_cols + ["PCIAT-PCIAT_Total", "sii"]:
    # sii_impute["sii"] = sii_impute["sii"].round().astype(int)
    if col == "sii":
        sii_impute[col] = sii_impute[col].round().astype(int)
    else:
        sii_impute[col] = sii_impute[col]
    train[col] = sii_impute[col]

with open("feature_imputer.pkl", "wb") as f:
    pickle.dump(feature_imputer, f)

train.head()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,PreInt_EduHx-Season,sii
0,5.0,0.0,51.0,16.877316,46.00,50.8,23.2,59.2,82.0,106.6,...,0.8,0.80,0.8,0.8,0.40,0.80,0.550,NaN,Fall,2
1,9.0,0.0,68.8,14.035590,48.00,46.0,22.0,75.0,70.0,122.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.000,Fall,Summer,0
2,10.0,1.0,71.0,16.648696,56.50,75.6,25.0,65.0,94.0,117.0,...,0.2,0.00,0.4,0.4,0.20,0.20,0.280,Fall,Summer,0
3,9.0,0.0,71.0,18.292347,56.00,81.6,26.0,60.0,97.0,117.0,...,0.0,0.60,0.8,0.6,0.80,0.20,0.440,Summer,Winter,1
4,18.0,1.0,68.2,26.713639,64.14,125.0,33.6,70.6,74.2,125.8,...,0.4,0.48,0.4,0.4,0.32,0.44,0.516,NaN,NaN,2


In [6]:
train["sii"].value_counts()

sii
0    2079
1    1390
2     457
3      34
Name: count, dtype: int64

### テーブルデータセット

In [7]:
add_features = [
    "BMI_Age",
    "Internet_Hours_Age",
    "BMI_Internet_Hours",
    "BFP_BMI",
    "FFMI_BFP",
    "FMI_BFP",
    "LST_TBW",
    "BFP_BMR",
    "BFP_DEE",
    "BMR_Weight",
    "DEE_Weight",
    "SMM_Height",
    "Muscle_to_Fat",
    "Hydration_Status",
    "ICW_TBW",
]


def feature_engineering(df):
    # season_cols = [col for col in df.columns if "Season" in col]
    # df = df.drop(season_cols, axis=1)
    df["BMI_Age"] = df["Physical-BMI"] * df["Basic_Demos-Age"]
    df["Internet_Hours_Age"] = (
        df["PreInt_EduHx-computerinternet_hoursday"] * df["Basic_Demos-Age"]
    )
    df["BMI_Internet_Hours"] = (
        df["Physical-BMI"] * df["PreInt_EduHx-computerinternet_hoursday"]
    )
    df["BFP_BMI"] = df["BIA-BIA_Fat"] / df["BIA-BIA_BMI"]
    df["FFMI_BFP"] = df["BIA-BIA_FFMI"] / df["BIA-BIA_Fat"]
    df["FMI_BFP"] = df["BIA-BIA_FMI"] / df["BIA-BIA_Fat"]
    df["LST_TBW"] = df["BIA-BIA_LST"] / df["BIA-BIA_TBW"]
    df["BFP_BMR"] = df["BIA-BIA_Fat"] * df["BIA-BIA_BMR"]
    df["BFP_DEE"] = df["BIA-BIA_Fat"] * df["BIA-BIA_DEE"]
    df["BMR_Weight"] = df["BIA-BIA_BMR"] / df["Physical-Weight"]
    df["DEE_Weight"] = df["BIA-BIA_DEE"] / df["Physical-Weight"]
    df["SMM_Height"] = df["BIA-BIA_SMM"] / df["Physical-Height"]
    df["Muscle_to_Fat"] = df["BIA-BIA_SMM"] / df["BIA-BIA_FMI"]
    df["Hydration_Status"] = df["BIA-BIA_TBW"] / df["Physical-Weight"]
    df["ICW_TBW"] = df["BIA-BIA_ICW"] / df["BIA-BIA_TBW"]

    return df


train = feature_engineering(train)
train = train.replace([np.inf, -np.inf], np.nan)
for add_ in add_features:
    train[add_] = train[add_].fillna(0.0)
train = train.dropna(thresh=10, axis=0)
test = feature_engineering(test)

In [8]:
# onehotEncoderの作成
from sklearn.preprocessing import OneHotEncoder

categorical_columns = [
    # "Basic_Demos-Enroll_Season",
    # "CGAS-Season",
    # "Physical-Season",
    # "PAQ_C-Season",
    # "FGC-Season",
    # "Fitness_Endurance-Season",
    # "PAQ_A-Season",
    # "BIA-Season",
    # "SDS-Season",
    "PreInt_EduHx-Season",
]

double_columns = [
    "FGC-FGC_SRR_Zone",
    "BIA-BIA_SMM",
    "Physical-Waist_Circumference",
    "BIA-BIA_FFMI",
    "FGC-FGC_CU",
    "PreInt_EduHx-computerinternet_hoursday",
    "BIA-BIA_ECW",
    "FGC-FGC_CU_Zone",
    "FGC-FGC_SRL_Zone",
    "BIA-BIA_DEE",
    "Physical-Weight",
    "Fitness_Endurance-Time_Mins",
    "FGC-FGC_SRR",
    "SDS-SDS_Total_T",
    "FGC-FGC_PU",
    "BIA-BIA_FFM",
    "FGC-FGC_TL_Zone",
    "Physical-BMI",
    "Physical-Systolic_BP",
    "Physical-HeartRate",
    "BIA-BIA_ICW",
    "Physical-Height",
    "FGC-FGC_SRL",
    "BIA-BIA_BMC",
    "Fitness_Endurance-Time_Sec",
    "BIA-BIA_Frame_num",
    "Basic_Demos-Age",
    "FGC-FGC_GSND_Zone",
    "Basic_Demos-Sex",
    "FGC-FGC_GSND",
    "BIA-BIA_LST",
    "FGC-FGC_TL",
    "BIA-BIA_BMI",
    "BIA-BIA_FMI",
    "PAQ_C-PAQ_C_Total",
    "BIA-BIA_Activity_Level_num",
    "FGC-FGC_GSD",
    "BIA-BIA_BMR",
    "BIA-BIA_Fat",
    "SDS-SDS_Total_Raw",
    "CGAS-CGAS_Score",
    "FGC-FGC_PU_Zone",
    "BIA-BIA_LDM",
    "Fitness_Endurance-Max_Stage",
    "PAQ_A-PAQ_A_Total",
    "BIA-BIA_TBW",
    "FGC-FGC_GSD_Zone",
    "Physical-Diastolic_BP",
]

###################### categorical columns ######################
# trainのtargetをonehot化
onehot_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
onehot_encoder.fit(train[categorical_columns])

with open("./assets/onehot_encoder.pkl", "wb") as f:
    pickle.dump(onehot_encoder, f)

categorical_feature = onehot_encoder.transform(train[categorical_columns])

###################### double columns ######################
# trainのtargetを標準化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train[double_columns + add_features])

with open("./assets/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

double_feature = scaler.transform(train[double_columns + add_features])
# double_feature = train[double_columns].values

# 欠損値の補完
double_feature = np.nan_to_num(double_feature)

###################### inputの作成 ######################

ids = train["id"].values.reshape(-1, 1)
X = np.concatenate([categorical_feature, double_feature], axis=1)

# DataFrameの作成
ids_df = pd.DataFrame(ids, columns=["id"])
X_df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
# y_df = pd.DataFrame(y, columns=["sii"])
y_df = train[survey_cols + ["PCIAT-PCIAT_Total", "sii"]]

train_df = pd.concat([ids_df, X_df, y_df], axis=1)
train_df

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,sii
0,00008ff9,1.0,0.0,0.0,0.0,0.0,-1.660420,-0.226383,-0.881999,-0.289421,...,0.80,0.80,0.80,0.80,0.80,0.80,0.40,0.80,0.550,2
1,000fd460,0.0,0.0,1.0,0.0,0.0,0.833611,-0.293529,-1.142337,-0.512530,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0
2,00105258,0.0,0.0,1.0,0.0,0.0,0.833611,-0.225184,-0.491493,-0.205461,...,0.20,0.20,0.20,0.00,0.40,0.40,0.20,0.20,0.280,0
3,00115b9f,0.0,0.0,0.0,1.0,0.0,-1.660420,-0.113593,-0.274545,-0.231795,...,0.60,0.00,0.00,0.60,0.80,0.60,0.80,0.20,0.440,1
4,0016bb22,0.0,0.0,0.0,0.0,1.0,-0.164001,0.228762,1.374260,0.205371,...,0.56,0.44,0.40,0.48,0.40,0.40,0.32,0.44,0.516,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,1.0,0.0,0.0,0.0,0.0,0.833611,-0.059962,-0.491493,-0.234290,...,0.40,0.00,0.20,0.00,0.40,0.20,0.20,0.00,0.320,1
3956,ffa9794a,0.0,0.0,0.0,1.0,0.0,-1.660420,-0.196234,-0.057597,-0.234016,...,0.08,0.08,0.28,0.16,0.40,0.16,0.16,0.08,0.224,0
3957,ffcd4dbd,1.0,0.0,0.0,0.0,0.0,0.833611,-0.076215,0.419689,-0.067594,...,0.20,0.60,0.00,0.00,0.20,0.20,0.00,0.20,0.310,1
3958,ffed1dd5,0.0,1.0,0.0,0.0,0.0,0.334805,0.194521,-0.057597,-0.457894,...,0.20,0.20,0.40,0.20,0.20,0.20,0.20,0.20,0.190,0


In [9]:
def read_parquet(base_dir, id_):
    path = os.path.join(base_dir, f"id={id_}", "part-0.parquet")
    return pd.read_parquet(path)


def get_valid_ids(base_dir):
    return [f.split("=")[1].split(".")[0] for f in os.listdir(base_dir)]


p = read_parquet(base_dir="../../inputs/series_train.parquet/", id_="ffcd4dbd")
# p = read_parquet(base_dir="../../inputs/series_train.parquet/", id_="10e46254")
# p

In [10]:
from glob import glob

# len(glob("../../normalized/*"))
len(glob("../../inputs/series_train.parquet/*"))

996

## Metric

In [11]:
from sklearn.metrics import *


def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")


def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(
        oof_non_rounded < thresholds[0],
        0,
        np.where(
            oof_non_rounded < thresholds[1],
            1,
            np.where(oof_non_rounded < thresholds[2], 2, 3),
        ),
    )


def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

## Model, Dataset

In [12]:
train_df.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,sii
0,00008ff9,1.0,0.0,0.0,0.0,0.0,-1.660420,-0.226383,-0.881999,-0.289421,...,0.80,0.80,0.8,0.80,0.8,0.8,0.40,0.80,0.550,2
1,000fd460,0.0,0.0,1.0,0.0,0.0,0.833611,-0.293529,-1.142337,-0.512530,...,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.000,0
2,00105258,0.0,0.0,1.0,0.0,0.0,0.833611,-0.225184,-0.491493,-0.205461,...,0.20,0.20,0.2,0.00,0.4,0.4,0.20,0.20,0.280,0
3,00115b9f,0.0,0.0,0.0,1.0,0.0,-1.660420,-0.113593,-0.274545,-0.231795,...,0.60,0.00,0.0,0.60,0.8,0.6,0.80,0.20,0.440,1
4,0016bb22,0.0,0.0,0.0,0.0,1.0,-0.164001,0.228762,1.374260,0.205371,...,0.56,0.44,0.4,0.48,0.4,0.4,0.32,0.44,0.516,2


In [13]:
from sklearn.model_selection import train_test_split

use_ids = list(
    train_df[train_df["sii"] != -1]["id"].unique()
)  # get_valid_ids(base_dir="../../normalized/")

len(use_ids)

3960

## Training

In [14]:
from tqdm import tqdm
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
use_ids = np.array(use_ids)
for train_index, valid_index in kf.split(use_ids):
    train_ids = [use_ids[i] for i in train_index]
    valid_ids = [use_ids[i] for i in valid_index]

    train_dataset = CMIDataset(
        table_df=train_df,
        valid_ids=use_ids,
        base_dir="../../inputs/series_train.parquet/",
        save_filename=nb_name,
    )

In [15]:
train_dataset[0]["time_input"].shape

torch.Size([31, 17280, 15])

In [16]:
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

CV = []

batch_size = 1

# use_ids = np.array(use_ids[:30]) # debug
use_ids = np.array(use_ids)

extract_df = train[train["id"].isin(use_ids)].reset_index(drop=True)

test_df = train[["id", "sii"]].copy()
# test_df["pred_sii"] = 0
oof_preds = []

for fold, (train_ids, valid_ids) in enumerate(
    skf.split(extract_df["id"], extract_df["sii"])
):
    print(f"################### fold:{fold} ###################")
    best_valid_score = -100

    train_ids = use_ids[train_ids]
    valid_ids = use_ids[valid_ids]

    train_dataset = CMIDataset(
        table_df=train_df,
        valid_ids=train_ids,
        base_dir="../../inputs/series_train.parquet/",
        save_filename=nb_name,
    )
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=30
    )

    vlaid_dataset = CMIDataset(
        table_df=train_df,
        valid_ids=valid_ids,
        base_dir="../../inputs/series_train.parquet/",
        save_filename=nb_name,
    )

    valid_loader = DataLoader(
        vlaid_dataset, batch_size=batch_size, shuffle=False, num_workers=30
    )
    # data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

    # model = TimeEncoder(input_size=26, hidden_size=13, num_layers=2).to("cuda")
    model = CMIModel(input_size=26, hidden_size=13, num_layers=2).to("cuda")

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    spot_oof_preds = []

    for epoch in range(5):
        total_train_loss = 0
        total_valid_loss = 0

        train_pred = []
        valid_pred = []
        trian_gt = []
        valid_gt = []

        tq = tqdm(train_loader)
        for i, data in enumerate(train_loader):
            table_input = data["table_input"].to("cuda")
            time_input = data["time_input"].to("cuda")
            mask = data["mask"].to("cuda").to(torch.float32)
            target_ = data["output"].to("cuda")
            sii_target = target_[:, :, -1]
            survey_target = target_[:, :, :21].squeeze(0)
            optimizer.zero_grad()
            sii_output, survey_output, attention_weight = model(
                table_input, time_input, active_mask=mask
            )
            loss = criterion(sii_output, sii_target) + criterion(
                survey_output, survey_target
            )
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

            train_pred.append(sii_output.detach().cpu().numpy())
            trian_gt.append(target_.detach().cpu().numpy())

            tq.set_postfix(loss=total_train_loss / (i + 1))
            tq.update()
        tq.close()

        tq = tqdm(valid_loader)
        for i, data in enumerate(valid_loader):
            table_input = data["table_input"].to("cuda")
            time_input = data["time_input"].to("cuda")
            mask = data["mask"].to("cuda").to(torch.float32)
            target_ = data["output"].to("cuda")
            sii_target = target_[:, :, -1]
            survey_target = target_[:, :, :21].squeeze(0)
            sii_output, survey_output, attention_weight = model(
                table_input, time_input, active_mask=mask
            )
            loss = criterion(sii_output, sii_target) + criterion(
                survey_output, survey_target
            )
            total_valid_loss += loss.item()

            valid_pred.append(sii_output.detach().cpu().numpy())
            valid_gt.append(target_.detach().cpu().numpy())

            tq.set_postfix(loss=total_valid_loss / (i + 1))
            tq.update()
        tq.close()

        metric_train_pred = np.concatenate(train_pred)
        metric_valid_pred = np.concatenate(valid_pred)
        metric_train_gt = np.concatenate(trian_gt)
        metric_valid_gt = np.concatenate(valid_gt)

        train_score = quadratic_weighted_kappa(
            metric_train_gt[:, :, -1], metric_train_pred.round(0).astype(int)
        )

        valid_score = quadratic_weighted_kappa(
            metric_valid_gt[:, :, -1], metric_valid_pred.round(0).astype(int)
        )

        print(
            f"epoch: {epoch}, loss: {total_train_loss / len(train_loader)}, valid_loss: {total_valid_loss / len(valid_loader)}, train_score: {train_score}, valid_score: {valid_score}"
        )

        if valid_score > best_valid_score:
            best_valid_score = valid_score
            torch.save(model.state_dict(), f"./assets/model_{fold}.pth")

            spot_oof_preds = []
            for i, id_ in enumerate(valid_ids):
                spot_oof_preds.append({"id": id_, "pred_sii": valid_pred[i][0][0]})

    oof_preds.append(spot_oof_preds)
    CV.append(best_valid_score)

print(f"CV: {np.mean(CV)}")

################### fold:0 ###################


100%|██████████| 990/990 [00:56<00:00, 17.53it/s, loss=0.479]


epoch: 0, loss: 0.5171990335367068, valid_loss: 0.47852584360957595, train_score: 0.3323653117524562, valid_score: 0.383182518290335


100%|██████████| 990/990 [00:23<00:00, 42.16it/s, loss=0.473]


epoch: 1, loss: 0.46406776052421456, valid_loss: 0.4734760436521034, train_score: 0.39142084343928185, valid_score: 0.39203009016324375


...

epoch: 2, loss: 0.4513807243667543, valid_loss: 0.4578063193562845, train_score: 0.39586357039187225, valid_score: 0.3728189324714649


100%|██████████| 990/990 [00:28<00:00, 34.45it/s, loss=0.499]


epoch: 3, loss: 0.4346179646616237, valid_loss: 0.4994636307597499, train_score: 0.4228863539425114, valid_score: 0.40916142916659204


100%|██████████| 990/990 [00:35<00:00, 28.15it/s, loss=0.469]


epoch: 4, loss: 0.4229166030614897, valid_loss: 0.46862504582786274, train_score: 0.45294912201710935, valid_score: 0.4201178187010237
CV: 0.39957008202502103


In [17]:
oof_preds_df = pd.concat([pd.DataFrame(p) for p in oof_preds], axis=0).reset_index(
    drop=True
)
oof_preds_df.head()

,id,pred_sii
0,0016bb22,0.942459
1,00ae59c9,0.864393
2,012cadd8,0.266893
3,014c303e,0.156301
4,021b2020,0.059649


In [18]:
test_pred_df = test_df.merge(oof_preds_df, on="id", how="inner")
test_pred_df.head()

,id,sii,pred_sii
0,00008ff9,2,0.459771
1,000fd460,0,0.250433
2,00105258,0,0.341704
3,00115b9f,1,0.201071
4,0016bb22,2,0.942459


In [19]:
from scipy.optimize import minimize

KappaOPtimizer = minimize(
    evaluate_predictions,
    x0=[0.5, 1.5, 2.5],
    args=(test_pred_df["sii"], test_pred_df["pred_sii"]),
    method="Nelder-Mead",
)
assert KappaOPtimizer.success, "Optimization did not converge."

oof_tuned = threshold_Rounder(test_pred_df["pred_sii"], KappaOPtimizer.x)
tKappa = quadratic_weighted_kappa(test_pred_df["sii"], oof_tuned)
print(f"tuned Kappa: {tKappa}")

tuned Kappa: 0.4455070917284827


In [20]:
print(KappaOPtimizer.x)

[0.50524349 1.0020262  3.0531382 ]


In [21]:
train

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,FMI_BFP,LST_TBW,BFP_BMR,BFP_DEE,BMR_Weight,DEE_Weight,SMM_Height,Muscle_to_Fat,Hydration_Status,ICW_TBW
0,5.0,0.0,51.0,16.877316,46.00,50.8,23.2,59.2,82.0,106.6,...,0.332267,1.190475,8591.822097,13746.944840,18.356260,29.370079,0.424811,6.383063,0.643522,0.747453
1,9.0,0.0,68.8,14.035590,48.00,46.0,22.0,75.0,70.0,122.0,...,0.305154,1.458119,3719.320478,5950.914352,20.362087,32.579348,0.321056,12.718037,0.588157,0.777492
2,10.0,1.0,71.0,16.648696,56.50,75.6,25.0,65.0,94.0,117.0,...,0.285049,1.209833,13767.473069,21724.485497,12.926463,20.397407,0.347170,4.884479,0.477630,0.657942
3,9.0,0.0,71.0,18.292347,56.00,81.6,26.0,60.0,97.0,117.0,...,0.224196,1.281264,21298.377749,36207.411592,13.865564,23.571569,0.472854,6.274343,0.563684,0.661008
4,18.0,1.0,68.2,26.713639,64.14,125.0,33.6,70.6,74.2,125.8,...,0.177312,1.328209,79057.011034,121546.184578,11.159904,17.157792,0.741197,4.731007,0.520795,0.543432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,13.0,0.0,60.0,16.362460,59.50,82.4,25.0,71.0,70.0,104.0,...,0.198595,1.254711,13985.687504,23775.715110,14.646602,24.899272,0.500487,12.939628,0.641165,0.631642
3956,10.0,0.0,58.6,18.764678,53.50,76.4,27.0,60.0,78.0,118.0,...,0.267664,1.238945,15914.063643,28114.560289,13.225976,23.365654,0.399926,5.075547,0.496013,0.665237
3957,11.0,0.0,68.0,21.441500,60.00,109.8,29.2,79.0,99.0,116.0,...,0.195299,1.310804,42623.022658,68197.040233,11.418397,18.269490,0.479653,4.334530,0.496020,0.623919
3958,13.0,0.0,70.0,12.235895,70.70,87.0,27.0,59.0,61.0,113.0,...,0.140658,1.270142,-8357.575498,-17550.944000,16.256782,34.139310,0.642631,-54.662704,0.780503,0.609266
